In [1]:
import ROOT
from ROOT import TMVA, TCut, TFile
from math import floor

Welcome to JupyROOT 6.22/03


In [2]:
outputFile = TFile( "TMVA_output.root", 'RECREATE' )
TMVA.Tools.Instance()

factory = TMVA.Factory( "TMVAClassification", outputFile,
                       "!V:Color:DrawProgressBar:Transformations=I;D;P;G,D:AnalysisType=Classification" )

In [3]:
factory.SetVerbose()

Ora dichiariamo un DataLoader e aggiungiamo le variabili usando la AddVariable Function

In [4]:
dataloader = TMVA.DataLoader("dataset")

In [5]:
#dataloader.AddVariable( "massK0S", 'F' )
dataloader.AddVariable( "bachelorPt", 'F')
dataloader.AddVariable( "combinedProtonProb", 'F')
dataloader.AddVariable( "signd0", 'F')
#dataloader.AddVariable( "tImpParV0", 'F')
dataloader.AddVariable( "tImpParBach", 'F' )
#dataloader.AddVariable( "CtK0S := DecayLengthK0S*0.497/v0P", 'F')
dataloader.AddVariable( "cosPAK0S", 'F' )
dataloader.AddVariable( "CosThetaStar", 'F' )
dataloader.AddVariable( "nSigmaTOFpr", 'F' )
dataloader.AddVariable( "nSigmaTPCpr", 'F' )
dataloader.AddVariable( "nSigmaTPCpi", 'F' )
#dataloader.AddVariable( "nSigmaTPCka", 'F' )
#dataloader.AddVariable( "bachelorEta", 'F' )
#dataloader.AddVariable( "LcPt", 'F' )
#spectator variables

dataloader.AddSpectator( "massLc2K0Sp", "mass Lc-->K0S+p", 'F' )
dataloader.AddSpectator( "massLc2Lambdapi",  "mass Lc -->L(1520)+pi", 'F' )
dataloader.AddSpectator( "massLambda", "mass V0 = Lambda", 'F' )
dataloader.AddSpectator( "massLambdaBar", "mass V0 = LambdaBar", 'F' )
dataloader.AddSpectator( "V0positivePt", "V0 positive Pt", 'F' )
dataloader.AddSpectator( "V0negativePt", "V0 negative Pt", 'F' )
dataloader.AddSpectator( "dcaV0pos", "dca V0 positive", 'F' )
dataloader.AddSpectator( "dcaV0neg", "dca V0 negative", 'F' )
dataloader.AddSpectator( "v0Pt", "K0S Pt", 'F' )
dataloader.AddSpectator( "dcaV0", "dca V0", 'F' )
dataloader.AddSpectator( "V0positiveEta", "V0pos Eta", 'F' )


apriamo il dataset contenente signal and background trees

In [6]:
input= TFile.Open( "AnalysisResults.root")

Warning in <TClass::Init>: no dictionary for class AliRDHFCutsLctoV0 is available
Warning in <TClass::Init>: no dictionary for class AliRDHFCuts is available
Warning in <TClass::Init>: no dictionary for class AliAnalysisCuts is available
Warning in <TClass::Init>: no dictionary for class AliVCuts is available
Warning in <TClass::Init>: no dictionary for class AliESDtrackCuts is available
Warning in <TClass::Init>: no dictionary for class AliAODPidHF is available
Warning in <TClass::Init>: no dictionary for class AliAnalysisUtils is available
Warning in <TClass::Init>: no dictionary for class AliEventCuts is available
Warning in <TClass::Init>: no dictionary for class AliNormalizationCounter is available
Warning in <TClass::Init>: no dictionary for class AliCounterCollection is available


In [7]:
# signal and backg trees for training
signal      = input.Get( "treeList_2_25_2_25_Sgn")
background  = input.Get( "treeList_2_25_2_25_Bkg")

global event weights per tree (see below for setting event-wise weights) Diamo al Training e al Test lo stesso tree, questo verrà quindi diviso in due sottounità che serviranno separatamente per il training e il test.

In [8]:
signalWeight     = 1.0
backgroundWeight = 1.0

dataloader.AddSignalTree(signal, signalWeight)
dataloader.AddBackgroundTree(background, backgroundWeight)

DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree treeList_2_25_2_25_Sgn of type Signal with 1762028 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree treeList_2_25_2_25_Bkg of type Background with 1473789 events


aggiungo dei Tagli sull'impulso trasverso per selezionare solo una parte degli eventi presenti nel tree

In [9]:
mycuts = TCut("LcPt<6 && LcPt>4")
mycutb = TCut("LcPt<6 && LcPt>4")

Applicato il taglio sul momento trasverso il numero di eventi da utilizzare nel training e nel test viene deciso come segue:

1) il training e il testing del segnale viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 100000

2) il training e il testing del Bakground viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 200000

Training e testing vengono ovviamente fatti con eventi diversi selezionati in maniera randomica all'interno del tree


In [10]:
nTrainingEventsSgn = floor(min(signal.GetEntries("LcPt<6 && LcPt>4")*0.5,100000))
nTrainingEventsBkg = floor(min(background.GetEntries("LcPt<6 && LcPt>4")*0.5,200000))
nTestingEventsSgn = floor(min(signal.GetEntries("LcPt<6 && LcPt>4")*0.5,100000))
nTestingEventsBkg = floor(min(background.GetEntries("LcPt<6 && LcPt>4")*0.5,200000))

Usiamo la funzione DataLoader.PrepareTrainingAndTestTree per applicare i cut agli eventi di input

In [11]:
dataloader.PrepareTrainingAndTestTree( mycuts, mycutb,
"nTrain_Signal=nTrainingEventsSgn:nTest_Signal=nTestingEventsSgn:nTrain_Background=nTrainingEventsBkg:nTest_Background=nTestingEventsBkg:SplitMode=Random:NormMode=NumEvents:!V")


In [12]:
factory.BookMethod( dataloader, TMVA.Types.kCuts, "Cuts",
                           "!H:!V:FitMethod=MC:EffSel:VarProp=FSmart" );

Factory                  : Booking method: Cuts
                         : 
                         : Use optimization method: "Monte Carlo"
                         : Use efficiency computation method: "Event Selection"
                         : Use "FSmart" cuts for variable: 'bachelorPt'
                         : Use "FSmart" cuts for variable: 'combinedProtonProb'
                         : Use "FSmart" cuts for variable: 'signd0'
                         : Use "FSmart" cuts for variable: 'tImpParBach'
                         : Use "FSmart" cuts for variable: 'cosPAK0S'
                         : Use "FSmart" cuts for variable: 'CosThetaStar'
                         : Use "FSmart" cuts for variable: 'nSigmaTOFpr'
                         : Use "FSmart" cuts for variable: 'nSigmaTPCpr'
                         : Use "FSmart" cuts for variable: 'nSigmaTPCpi'


In [13]:
#factory.BookMethod( dataloader, TMVA.Types.kBDT, "BDT",
#"!H:!V:NTrees=850:MinNodeSize=2.5%:MaxDepth=3:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" )

In [14]:
#factory.BookMethod(dataloader, TMVA.Types.kMLP, "MLP", 
#                   "H:!V:NeuronType=tanh:VarTransform=N:NCycles=600:HiddenLayers=N+5:TestRate=5:!UseRegulator" )

In [15]:
# Train MVAs
factory.TrainAllMethods()

Factory                  : Train all methods
                         : Building event vectors for type 2 Signal
                         : Dataset[dataset] :  create input formulas for tree treeList_2_25_2_25_Sgn
                         : Building event vectors for type 2 Background
                         : Dataset[dataset] :  create input formulas for tree treeList_2_25_2_25_Bkg
DataSetFactory           : [dataset] : Number of events in input trees
                         : Dataset[dataset] :     Signal     requirement: "LcPt<6 && LcPt>4"
                         : Dataset[dataset] :     Signal          -- number of events passed: 499411  / sum of weights: 499411
                         : Dataset[dataset] :     Signal          -- efficiency             : 0.28343
                         : Dataset[dataset] :     Background requirement: "LcPt<6 && LcPt>4"
                         : Dataset[dataset] :     Background      -- number of events passed: 252957  / sum of weights: 252957


0%, time left: unknown
7%, time left: 5 mins
13%, time left: 271 sec
19%, time left: 245 sec
25%, time left: 229 sec
32%, time left: 205 sec
38%, time left: 187 sec
44%, time left: 168 sec
50%, time left: 150 sec
57%, time left: 129 sec
63%, time left: 110 sec
69%, time left: 92 sec
75%, time left: 74 sec
82%, time left: 53 sec
88%, time left: 36 sec
94%, time left: 18 sec
0%, time left: unknown
6%, time left: 0 sec
12%, time left: 0 sec
18%, time left: 0 sec
25%, time left: 0 sec
31%, time left: 0 sec
37%, time left: 0 sec
43%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
62%, time left: 0 sec
68%, time left: 0 sec
75%, time left: 0 sec
81%, time left: 0 sec
87%, time left: 0 sec
93%, time left: 0 sec


In [16]:
# Test MVAs
factory.TestAllMethods()

Factory                  : Test all methods
Factory                  : Test method: Cuts for Classification performance
                         : 
Cuts                     : [dataset] : Evaluation of Cuts on testing sample (376183 events)
                         : Elapsed time for evaluation of 376183 events: 0.0314 sec       


0%, time left: unknown
6%, time left: 0 sec
12%, time left: 0 sec
18%, time left: 0 sec
25%, time left: 0 sec
31%, time left: 0 sec
37%, time left: 0 sec
43%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
62%, time left: 0 sec
68%, time left: 0 sec
75%, time left: 0 sec
81%, time left: 0 sec
87%, time left: 0 sec
93%, time left: 0 sec


In [17]:
# Evaluate MVAs
factory.EvaluateAllMethods() 

Factory                  : Evaluate all methods
Factory                  : Evaluate classifier: Cuts
                         : 
<WARNING>                : You have asked for histogram MVA_EFF_BvsS which does not seem to exist in *Results* .. better don't use it 
<WARNING>                : You have asked for histogram EFF_BVSS_TR which does not seem to exist in *Results* .. better don't use it 
<WARNING>                :  unable to fill 41 efficiency bins 
TFHandler_Cuts           :           Variable                  Mean                  RMS          [        Min                  Max ]
                         : -------------------------------------------------------------------------------------------------------------
                         :         bachelorPt:              2.3635              1.1013   [             0.34172              5.7675 ]
                         : combinedProtonProb:             0.68777             0.35063   [          4.6705e-05              1.0000 ]
  

In [18]:
# Save the output.
outputFile.Close()